<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Stock_Analysis/Nifty100_Upside_Analysis_Forecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance
#!pip install talib-binary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import numpy as np

mypf = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Data-Science-with-Python/master/Stock_Analysis/data/Nifty100Scrips.csv')

mypf = mypf[mypf['InScope'] == 1]

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol']

#stock_ids

stock_ids = stock_n100

In [3]:
mypf = mypf[mypf.Forecast.notnull()]
stock_ids = mypf[mypf.Forecast.notnull()]['Symbol'].unique()

In [4]:
stock_ids.sort()
stock_ids

array(['ASIANPAINT', 'AXISBANK', 'BAJAJFINSV', 'BAJFINANCE', 'BERGEPAINT',
       'BHARTIARTL', 'BRITANNIA', 'CANBK', 'CIPLA', 'COLPAL', 'DABUR',
       'DEEPAKNTR', 'DIVISLAB', 'DMART', 'DRREDDY', 'EICHERMOT',
       'FEDERALBNK', 'GODREJCP', 'GODREJIND', 'HAVELLS', 'HCLTECH',
       'HDFC', 'HDFCBANK', 'HDFCLIFE', 'HEROMOTOCO', 'HINDUNILVR',
       'IBREALEST', 'ICICIBANK', 'ICICIPRULI', 'INDIGO', 'INDUSINDBK',
       'INFY', 'IRCTC', 'ITC', 'JUBLFOOD', 'KOTAKBANK', 'LT', 'LTIM',
       'M&M', 'MARUTI', 'MCDOWELL-N', 'MOTILALOFS', 'MUTHOOTFIN',
       'NATIONALUM', 'NAUKRI', 'PIDILITIND', 'RELIANCE', 'SAIL',
       'SBILIFE', 'SBIN', 'SUNPHARMA', 'TATACONSUM', 'TATAMOTORS',
       'TATAPOWER', 'TATASTEEL', 'TCS', 'TECHM', 'TITAN', 'ULTRACEMCO',
       'WIPRO', 'ZEEL'], dtype=object)

In [5]:
mypf['InPortfolio'].value_counts()

0    51
1    10
Name: InPortfolio, dtype: int64

In [6]:
mypf[(mypf['Symbol'] + '.NS') == (stock_ids[1] + '.NS')]['Forecast'].values[0]

1060.0

In [7]:
# import necessary libraries 

%matplotlib inline
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib 
yfin.pdr_override()

In [8]:
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
    #end_date = '2020-01-01'
    end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
    short_window = 20 
    mid_window = 50
    long_window = 200
    moving_avg = 'EMA'


    # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-'))) 
    stock_df = web.DataReader(stock_symbol,  start = start, end = end)['Close']
    stock_df = pd.DataFrame(stock_df) # convert Series object to dataframe 
    stock_df.columns = {'Close Price'} # assign new colun name
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows 

    # column names for long and short moving average columns
    short_window_col = str(short_window) + '_' + moving_avg
    mid_window_col = str(mid_window) + '_' + moving_avg
    long_window_col = str(long_window) + '_' + moving_avg  

    # Create short exponential moving average column
    stock_df[short_window_col] = round(stock_df['Close Price'].ewm(span = short_window, adjust = False).mean(),0)

    stock_df[mid_window_col] = round(stock_df['Close Price'].ewm(span = mid_window, adjust = False).mean(),0)     

    #stock_df['RSI'] = round(talib.RSI(stock_df['Close Price'], timeperiod=14),0)

    # Create a long exponential moving average column
    stock_df[long_window_col] = round(stock_df['Close Price'].ewm(span = long_window, adjust = False).mean(),0)
    stock_df['Stock'] = stock_symbol
    stock_df['Dev%_200'] = round((stock_df['Close Price'] - stock_df['200_EMA'])*100/stock_df['200_EMA'],2)
    stock_df['200_EMA_Lag10'] = stock_df['200_EMA'].shift(10)
    stock_df['Slope%_200'] = round((stock_df['200_EMA'] - stock_df['200_EMA_Lag10'])*100/stock_df['200_EMA'],2)
    stock_df['50_EMA_Lag5'] = stock_df['50_EMA'].shift(5)
    stock_df['Slope%_50'] = round((stock_df['50_EMA'] - stock_df['50_EMA_Lag5'])*100/stock_df['50_EMA'],2)
    stock_df['Max'] = round(max(stock_df['Close Price']),0)
    stock_df['Min'] = round(min(stock_df['Close Price']),0)    
    #stock_df['Dev%_Max'] = round((stock_df['Close Price'] - stock_df['Max'])*100/stock_df['Max'],2)
    #stock_df['Dev%_Min'] = round((stock_df['Close Price'] - stock_df['Min'])*100/stock_df['Min'],2)
    stock_df['Premium%'] = round((stock_df['Close Price']-stock_df['Min'])*100/(stock_df['Max']-stock_df['Min']),2)
    stock_df['Discount%'] = round((stock_df['Max']-stock_df['Close Price'])*100/(stock_df['Max']-stock_df['Min']),2)
    stock_df['Close Price'] = round(stock_df['Close Price'],0) 
    stock_df.drop(['20_EMA','50_EMA','200_EMA','200_EMA_Lag10','50_EMA_Lag5'], axis=1, inplace=True)
    
    
    return stock_df

df_prec_dev = pd.DataFrame()

#tmp = stock_prec_dev('ULTRACEMCO.NS')
#df_perform = df_perform.append(tmp)
for stock_id in stock_ids:
    stock_id = stock_id.upper() + '.NS'
    print(stock_id)
    tmp = stock_prec_dev(stock_id)
    tmp = tmp.reset_index()
    tmp = tmp.tail(1)
    tmp['Forecast'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Forecast'].values[0]
    df_prec_dev = df_prec_dev.append(tmp)

ASIANPAINT.NS
[*********************100%***********************]  1 of 1 completed
AXISBANK.NS
[*********************100%***********************]  1 of 1 completed
BAJAJFINSV.NS
[*********************100%***********************]  1 of 1 completed
BAJFINANCE.NS
[*********************100%***********************]  1 of 1 completed
BERGEPAINT.NS
[*********************100%***********************]  1 of 1 completed
BHARTIARTL.NS
[*********************100%***********************]  1 of 1 completed
BRITANNIA.NS
[*********************100%***********************]  1 of 1 completed
CANBK.NS
[*********************100%***********************]  1 of 1 completed
CIPLA.NS
[*********************100%***********************]  1 of 1 completed
COLPAL.NS
[*********************100%***********************]  1 of 1 completed
DABUR.NS
[*********************100%***********************]  1 of 1 completed
DEEPAKNTR.NS
[*********************100%***********************]  1 of 1 completed
DIVISLAB.NS
[**************

In [9]:
stock_prec_dev('^NSEI').tail(5)

[*********************100%***********************]  1 of 1 completed


,Close Price,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%
Date,,,,,,,,,
2022-12-15,18415.0,^NSEI,5.74,0.71,0.45,18812.0,15294.0,88.71,11.29
2022-12-16,18269.0,^NSEI,4.84,0.68,0.38,18812.0,15294.0,84.57,15.43
2022-12-19,18420.0,^NSEI,5.65,0.65,0.35,18812.0,15294.0,88.87,11.13
2022-12-20,18385.0,^NSEI,5.40,0.63,0.29,18812.0,15294.0,87.87,12.13
2022-12-21,18451.0,^NSEI,5.71,0.62,0.23,18812.0,15294.0,89.73,10.27


In [10]:
df_prec_dev['Upside%'] = round((df_prec_dev['Forecast']-df_prec_dev['Close Price'])*100/(df_prec_dev['Close Price']),2)
df_prec_dev = df_prec_dev.sort_values(by = 'Upside%', ascending=False)
stock_ids = df_prec_dev['Stock'].values
df_prec_dev.reset_index(drop=True, inplace=True)

In [11]:
# heavy upside
df_prec_dev[df_prec_dev['Upside%'] > 20].sort_values(by = 'Upside%', ascending=False)

,Date,Close Price,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,Forecast,Upside%
0,2022-12-21,458.0,ICICIPRULI.NS,-11.44,-1.35,-1.45,616.0,444.0,8.05,91.95,640.0,39.74
1,2022-12-21,460.0,GODREJIND.NS,-2.45,0.00,0.67,644.0,405.0,23.20,76.80,638.0,38.70
2,2022-12-21,1239.0,SBILIFE.NS,2.12,0.41,-0.08,1332.0,1025.0,69.63,30.37,1607.0,29.70
3,2022-12-21,724.0,MOTILALOFS.NS,-5.29,-1.05,0.58,971.0,654.0,22.24,77.76,930.0,28.45
4,2022-12-21,528.0,JUBLFOOD.NS,-9.16,-1.03,-1.25,786.0,464.0,19.81,80.19,643.0,21.78
5,2022-12-21,6677.0,BAJFINANCE.NS,-1.82,-0.29,-0.63,7853.0,5284.0,54.22,45.78,8015.0,20.04


In [12]:
# near 52W low
df_prec_dev[df_prec_dev['Discount%'] > 90].sort_values(by = 'Upside%', ascending=False)

,Date,Close Price,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,Forecast,Upside%
0,2022-12-21,458.0,ICICIPRULI.NS,-11.44,-1.35,-1.45,616.0,444.0,8.05,91.95,640.0,39.74
20,2022-12-21,1026.0,TECHM.NS,-11.21,-1.04,-0.66,1806.0,965.0,7.30,92.70,1139.0,11.01
33,2022-12-21,392.0,WIPRO.NS,-13.65,-1.32,-0.25,722.0,375.0,4.91,95.09,425.0,8.42
43,2022-12-21,3380.0,DIVISLAB.NS,-9.52,-1.10,-0.50,4678.0,3266.0,8.11,91.89,3560.0,5.33


In [13]:
# buy or accumulate (based on std dev from 200 EMA)
df_prec_dev[df_prec_dev['Dev%_200'] < -10].sort_values(by = 'Dev%_200', ascending=True)

,Date,Close Price,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,Forecast,Upside%
33,2022-12-21,392.0,WIPRO.NS,-13.65,-1.32,-0.25,722.0,375.0,4.91,95.09,425.0,8.42
0,2022-12-21,458.0,ICICIPRULI.NS,-11.44,-1.35,-1.45,616.0,444.0,8.05,91.95,640.0,39.74
20,2022-12-21,1026.0,TECHM.NS,-11.21,-1.04,-0.66,1806.0,965.0,7.30,92.70,1139.0,11.01
44,2022-12-21,4428.0,LTIM.NS,-10.69,-1.21,-1.38,7562.0,3824.0,16.16,83.84,4654.0,5.10


In [14]:
# sell or downsize (based on std dev from 200 EMA)
df_prec_dev[df_prec_dev['Dev%_200'] > 10].sort_values(by = 'Dev%_200', ascending=False)

,Date,Close Price,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,Forecast,Upside%
56,2022-12-21,324.0,CANBK.NS,29.28,2.79,1.67,329.0,175.0,97.08,2.92,310.0,-4.32
21,2022-12-21,956.0,AXISBANK.NS,21.78,2.17,1.59,956.0,626.0,100.00,0.00,1060.0,10.88
26,2022-12-21,136.0,FEDERALBNK.NS,19.08,1.75,0.76,138.0,81.0,96.05,3.95,149.0,9.56
59,2022-12-21,4503.0,BRITANNIA.NS,18.37,1.79,1.65,4526.0,3094.0,98.38,1.62,3935.0,-12.61
17,2022-12-21,1233.0,INDUSINDBK.NS,16.32,1.60,1.02,1264.0,778.0,93.62,6.38,1393.0,12.98
58,2022-12-21,2157.0,LT.NS,14.17,1.59,1.27,2195.0,1472.0,94.70,5.30,2036.0,-5.61
10,2022-12-21,607.0,SBIN.NS,13.00,1.49,0.68,626.0,435.0,89.95,10.05,710.0,16.97
13,2022-12-21,1281.0,M&M.NS,11.89,1.22,0.16,1354.0,691.0,89.01,10.99,1468.0,14.60
51,2022-12-21,340.0,ITC.NS,11.83,1.32,0.00,361.0,208.0,86.24,13.76,350.0,2.94
45,2022-12-21,2071.0,INDIGO.NS,11.05,0.91,1.42,2265.0,1549.0,72.91,27.09,2176.0,5.07


In [15]:
# portfolio buy/sell
mypf['Stock'] = mypf['Symbol'] + '.NS'
df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)].sort_values(by = 'Upside%', ascending=False)

,Date,Close Price,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,Forecast,Upside%
0,2022-12-21,458.0,ICICIPRULI.NS,-11.44,-1.35,-1.45,616.0,444.0,8.05,91.95,640.0,39.74
2,2022-12-21,1239.0,SBILIFE.NS,2.12,0.41,-0.08,1332.0,1025.0,69.63,30.37,1607.0,29.70
3,2022-12-21,724.0,MOTILALOFS.NS,-5.29,-1.05,0.58,971.0,654.0,22.24,77.76,930.0,28.45
4,2022-12-21,528.0,JUBLFOOD.NS,-9.16,-1.03,-1.25,786.0,464.0,19.81,80.19,643.0,21.78
12,2022-12-21,589.0,HDFCLIFE.NS,4.03,0.35,0.71,674.0,504.0,49.88,50.12,681.0,15.62
16,2022-12-21,1515.0,INFY.NS,-3.65,-0.19,-0.45,1940.0,1365.0,26.01,73.99,1715.0,13.20
20,2022-12-21,1026.0,TECHM.NS,-11.21,-1.04,-0.66,1806.0,965.0,7.30,92.70,1139.0,11.01
24,2022-12-21,1109.0,MUTHOOTFIN.NS,-2.20,-0.26,0.28,1540.0,956.0,26.21,73.79,1217.0,9.74
27,2022-12-21,4271.0,NAUKRI.NS,1.98,-0.12,0.89,5696.0,3375.0,38.60,61.40,4677.0,9.51
43,2022-12-21,3380.0,DIVISLAB.NS,-9.52,-1.10,-0.50,4678.0,3266.0,8.11,91.89,3560.0,5.33


In [16]:
df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]['Upside%']*100/df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]['Upside%'].sum()

0     21.588440
2     16.134289
3     15.455237
4     11.831812
12     8.485441
16     7.170795
20     5.981095
24     5.291178
27     5.166232
43     2.895480
Name: Upside%, dtype: float64

In [17]:
# prospects buy
tmp = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)].sort_values(by = 'Upside%', ascending=False)
tmp[tmp['Upside%'] > 20].sort_values(by = 'Upside%', ascending=False)

,Date,Close Price,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,Forecast,Upside%
1,2022-12-21,460.0,GODREJIND.NS,-2.45,0.00,0.67,644.0,405.0,23.20,76.80,638.0,38.70
5,2022-12-21,6677.0,BAJFINANCE.NS,-1.82,-0.29,-0.63,7853.0,5284.0,54.22,45.78,8015.0,20.04


In [18]:
# CCI buy
tmp = df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['CCIFlag'] == 1].Stock.values)].sort_values(by = 'Upside%', ascending=False)
tmp[tmp['Upside%'] > 20].sort_values(by = 'Upside%', ascending=False)

,Date,Close Price,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,Forecast,Upside%
4,2022-12-21,528.0,JUBLFOOD.NS,-9.16,-1.03,-1.25,786.0,464.0,19.81,80.19,643.0,21.78


In [19]:
# turning around 
df_prec_dev[(df_prec_dev['Slope%_200'] > 0) & (df_prec_dev['Slope%_200'] < 1)].sort_values(by = 'Slope%_200', ascending=True)

,Date,Close Price,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,Forecast,Upside%
19,2022-12-21,1850.0,KOTAKBANK.NS,0.35,0.11,-0.42,1966.0,1642.0,64.35,35.65,2055.0,11.08
25,2022-12-21,8535.0,MARUTI.NS,0.36,0.12,-0.69,9528.0,6769.0,64.02,35.98,9356.0,9.62
23,2022-12-21,4372.0,DRREDDY.NS,0.39,0.14,-0.11,4907.0,3719.0,54.97,45.03,4829.0,10.45
49,2022-12-21,1625.0,BAJAJFINSV.NS,3.02,0.19,-0.43,1837.0,1093.0,71.47,28.53,1691.0,4.06
18,2022-12-21,2115.0,DEEPAKNTR.NS,-0.33,0.19,-0.51,2660.0,1720.0,42.02,57.98,2371.0,12.10
53,2022-12-21,1600.0,COLPAL.NS,2.25,0.26,-0.13,1678.0,1394.0,72.62,27.38,1606.0,0.38
9,2022-12-21,2519.0,TITAN.NS,1.36,0.28,-0.73,2770.0,1936.0,69.88,30.12,2967.0,17.78
30,2022-12-21,2624.0,RELIANCE.NS,3.78,0.32,0.12,2820.0,2236.0,66.35,33.65,2854.0,8.77
12,2022-12-21,589.0,HDFCLIFE.NS,4.03,0.35,0.71,674.0,504.0,49.88,50.12,681.0,15.62
57,2022-12-21,2587.0,PIDILITIND.NS,1.30,0.39,-0.64,2902.0,1999.0,65.14,34.86,2448.0,-5.37


In [20]:
# turning around
df_prec_dev[(df_prec_dev['Slope%_200'] < 0) & (df_prec_dev['Slope%_50'] > 0)].sort_values(by = 'Upside%', ascending=False)

,Date,Close Price,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,Forecast,Upside%
3,2022-12-21,724.0,MOTILALOFS.NS,-5.29,-1.05,0.58,971.0,654.0,22.24,77.76,930.0,28.45
24,2022-12-21,1109.0,MUTHOOTFIN.NS,-2.20,-0.26,0.28,1540.0,956.0,26.21,73.79,1217.0,9.74
27,2022-12-21,4271.0,NAUKRI.NS,1.98,-0.12,0.89,5696.0,3375.0,38.60,61.40,4677.0,9.51


In [21]:
# bull run
df_prec_dev[(df_prec_dev['Slope%_200'] > 0) & (df_prec_dev['Slope%_50'] > df_prec_dev['Slope%_200'])].sort_values(by = 'Slope%_200', ascending=False)

,Date,Close Price,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,Forecast,Upside%
45,2022-12-21,2071.0,INDIGO.NS,11.05,0.91,1.42,2265.0,1549.0,72.91,27.09,2176.0,5.07
36,2022-12-21,894.0,GODREJCP.NS,4.18,0.47,0.69,968.0,665.0,75.53,24.47,958.0,7.16
12,2022-12-21,589.0,HDFCLIFE.NS,4.03,0.35,0.71,674.0,504.0,49.88,50.12,681.0,15.62


In [22]:
# about to turn around
df_prec_dev[(df_prec_dev['Slope%_200'] < 0) & (df_prec_dev['Slope%_50'] > df_prec_dev['Slope%_200'])].sort_values(by = 'Slope%_200', ascending=False)

,Date,Close Price,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,Forecast,Upside%
27,2022-12-21,4271.0,NAUKRI.NS,1.98,-0.12,0.89,5696.0,3375.0,38.60,61.40,4677.0,9.51
46,2022-12-21,4086.0,DMART.NS,-1.54,-0.17,-0.12,4731.0,3231.0,57.02,42.98,4282.0,4.80
24,2022-12-21,1109.0,MUTHOOTFIN.NS,-2.20,-0.26,0.28,1540.0,956.0,26.21,73.79,1217.0,9.74
20,2022-12-21,1026.0,TECHM.NS,-11.21,-1.04,-0.66,1806.0,965.0,7.30,92.70,1139.0,11.01
3,2022-12-21,724.0,MOTILALOFS.NS,-5.29,-1.05,0.58,971.0,654.0,22.24,77.76,930.0,28.45
43,2022-12-21,3380.0,DIVISLAB.NS,-9.52,-1.10,-0.50,4678.0,3266.0,8.11,91.89,3560.0,5.33
14,2022-12-21,79.0,NATIONALUM.NS,-5.36,-1.20,0.00,130.0,67.0,18.33,81.67,90.0,13.92
33,2022-12-21,392.0,WIPRO.NS,-13.65,-1.32,-0.25,722.0,375.0,4.91,95.09,425.0,8.42
